<a href="https://colab.research.google.com/github/snowywhitney/ML_Examples/blob/main/NLP_NER_transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import os

# get current directory
cwd = os.getcwd()

# import AG Databet
data = pd.read_csv('data/train.csv')
data = pd.DataFrame(data=data)

data.columns = data.columns.str.replace(" ","_")
data.columns = data.columns.str.lower()

data['class_name'] = data['class_index'].map({1: "World", 2: "Sports", 3: "Business", 4: "Sci_Tech"})

In [3]:
data.head()

,class_index,title,description,class_name
0,3,Wall St. Bears Claw Back Into the Black (Reuters),"Reuters - Short-sellers, Wall Street's dwindli...",Business
1,3,Carlyle Looks Toward Commercial Aerospace (Reu...,Reuters - Private investment firm Carlyle Grou...,Business
2,3,Oil and Economy Cloud Stocks' Outlook (Reuters),Reuters - Soaring crude prices plus worries\ab...,Business
3,3,Iraq Halts Oil Exports from Main Southern Pipe...,Reuters - Authorities have halted oil export\f...,Business
4,3,"Oil prices soar to all-time record, posing new...","AFP - Tearaway world oil prices, toppling reco...",Business


In [4]:
data.shape

(120000, 4)

In [5]:
data.class_name.value_counts()

Business    30000
Sci_Tech    30000
Sports      30000
World       30000
Name: class_name, dtype: int64

In [6]:
cols = ['title', 'description']
data[cols] = data[cols].applymap(lambda x: x.replace("\\",  " "))
data[cols] = data[cols].applymap(lambda x: x.replace("#36;", "$"))
data[cols] = data[cols].applymap(lambda x: x.replace("  ", " "))
data[cols] = data[cols].applymap(lambda x: x.strip())

In [17]:
!pip install -U spacy[lookups,transformers]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 7.2 MB/s eta 0:00:00
  Using cached spacy_lookups_data-1.0.5-py2.py3-none-any.whl (98.5 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.9/197.9 kB 4.6 MB/s eta 0:00:00
  Using cached spacy_alignments-0.9.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (313 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 4.4 MB/s eta 0:00:00
  Attempting uninstall: spacy
    Found existing installation: spacy 3.6.1
    Uninstalling spacy-3.6.1:
      Successfully uninstalled spacy-3.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
en-core-web-sm 3.6.0 requires spacy<3.7.0,>=3.6.0, but you have spacy 3.7.2 which is incompatible.


In [18]:
!pip install -U spacy-lookups-data==1.0.0
!pip install cupy-cuda110==8.5.0
!python -m spacy download en_core_web_trf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.4/93.4 MB 5.2 MB/s eta 0:00:00
  Attempting uninstall: spacy-lookups-data
    Found existing installation: spacy-lookups-data 1.0.5
    Uninstalling spacy-lookups-data-1.0.5:
      Successfully uninstalled spacy-lookups-data-1.0.5
ERROR: Could not find a version that satisfies the requirement cupy-cuda110==8.5.0 (from versions: 10.0.0, 10.1.0, 10.2.0, 10.3.0, 10.3.1, 10.4.0, 10.5.0, 10.6.0, 11.0.0, 11.1.0, 11.2.0, 11.3.0, 11.4.0, 11.5.0, 11.6.0, 12.0.0, 12.1.0, 12.2.0, 12.3.0)
ERROR: No matching distribution found for cupy-cuda110==8.5.0
2024-01-20 07:54:19.744113: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-20 07:54:19.744167: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already be

In [19]:
import spacy

In [20]:
spacy.require_gpu()

True

In [21]:
nlp = spacy.load('en_core_web_trf')

In [22]:
import pprint
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(nlp.meta)

{   'author': 'Explosion',
    'components': [   'transformer',
                      'tagger',
                      'parser',
                      'attribute_ruler',
                      'lemmatizer',
                      'ner'],
    'description': 'English transformer pipeline '
                   "(Transformer(name='roberta-base', "
                   "piece_encoder='byte-bpe', stride=104, type='roberta', "
                   'width=768, window=144, vocab_size=50265)). Components: '
                   'transformer, tagger, parser, ner, attribute_ruler, '
                   'lemmatizer.',
    'disabled': [],
    'email': 'contact@explosion.ai',
    'labels': {   'attribute_ruler': [],
                  'lemmatizer': [],
                  'ner': [   'CARDINAL',
                             'DATE',
                             'EVENT',
                             'FAC',
                             'GPE',
                             'LANGUAGE',
                             'LAW',

In [23]:
for i in range(9):
  print("Article", i)
  print(data.loc[i, 'description'])
  print("Text Start End Label")
  doc = nlp(data.loc[i, "description"])
  for token in doc.ents:
    print(token.text, token.start_char, token.end_char, token.label_)
  print('\n')

Article 0
Reuters - Short-sellers, Wall Street's dwindling band of ultra-cynics, are seeing green again.
Text Start End Label
Reuters 0 7 ORG


Article 1
Reuters - Private investment firm Carlyle Group, which has a reputation for making well-timed and occasionally controversial plays in the defense industry, has quietly placed its bets on another part of the market.
Text Start End Label
Reuters 0 7 ORG
Carlyle Group 34 47 ORG


Article 2
Reuters - Soaring crude prices plus worries about the economy and the outlook for earnings are expected to hang over the stock market next week during the depth of the summer doldrums.
Text Start End Label
Reuters 0 7 ORG
next week 134 143 DATE
summer 168 174 DATE


Article 3
Reuters - Authorities have halted oil export flows from the main pipeline in southern Iraq after intelligence showed a rebel militia could strike infrastructure, an oil official said on Saturday.
Text Start End Label
Reuters 0 7 ORG
Iraq 86 90 GPE
Saturday 186 194 DATE


Article 4